P1 continuous FEM, stationary linear elliptic ESV2007 problem
================================

This example is about approximating the solution $u$ of the elliptic problem

$$\begin{align}
  -\nabla\cdot( \kappa \nabla u ) &= f   &&\text{in } \Omega\\
                                u &= g_D &&\text{on }\partial\Omega
\end{align}$$

with datafunction as defined in `dune/gdt/test/linearelliptic/problems/ESV2007.hh` (see below) using piecewise linear continuous Finite Elements, as in `dune/gdt/test/linearelliptic/discretizers/cg.hh`.

Note that the discretization below contains handling of arbitrary Dirichlet and Neumann boundary data, although the problem at hand contains only trivial Dirichlet data.

In [ ]:
from dune.xt import common, grid, functions, la
from dune import gdt
common.init_mpi()

$$\begin{align}
  \Omega &= [-1, 1]^2\\
  \Gamma_D &= \partial\Omega\\
  \Gamma_N &= \emptyset
\end{align}$$

In [ ]:
g = grid.make_cube_grid__2d_simplex_aluconform(lower_left=[-1, -1],
                                               upper_right=[1, 1],
                                               num_elements=[4, 4],
                                               num_refinements=2,
                                               overlap_size=[0, 0])
#g.visualize('../cgfem_esv2007_grid')
boundary_info = grid.make_boundary_info_on_leaf_layer(g, 'xt.grid.boundaryinfo.alldirichlet')
apply_on_neumann_boundary = grid.make_apply_on_neumann_intersections_leaf_part(boundary_info)
apply_on_dirichlet_boundary = grid.make_apply_on_dirichlet_intersections_leaf_part(boundary_info)

$$\begin{align}\kappa(x) &:= 1\\
f(x) &:= \tfrac{1}{2} \pi^2 \cos(\tfrac{1}{2} \pi x_0) \cos(\tfrac{1}{2} \pi x_1)\\
g_D(x) &:= 0\end{align}$$

Note that the grid `g` is only provided to select the correct _type_ of function. These functions do not rely on the actual grid which is why we need to later on provide the grid again, i.e., for `visualize(g)`.

In [ ]:
kappa = functions.make_constant_function_1x1(g, 1.0, name='diffusion')
f = functions.make_expression_function_1x1(g,
                                           'x',
                                           '0.5*pi*pi*cos(0.5*pi*x[0])*cos(0.5*pi*x[1])',
                                           order=3,
                                           name='force')
g_D = functions.make_constant_function_1x1(g, 0.0, name='dirichlet')
g_N = functions.make_constant_function_1x1(g, 0.0, name='neumann')
#kappa.visualize(g, '../cgfem_esv2007_diffusion')
#f.visualize(g, '../cgfem_esv2007_force')

In [ ]:
space = gdt.make_cg_leaf_part_to_1x1_fem_p1_space(g)
#space.visualize('../cgfem_esv2007_cg_space')

# There are two ways to create containers: 
# * manually create them and given them to the operators/functionals
# * let those create appropriate ones

# in the SWIPDG example we chose the former, so here we do the latterfor the lhs by not providing a matrix, only the type of container
elliptic_operator = gdt.make_elliptic_matrix_operator_istl_row_major_sparse_matrix_double(kappa, space)

# for the rhs, we manually create a vector which is provided to all functionals to assemble into
rhs_vector = la.IstlDenseVectorDouble(space.size(), 0.0)

l2_force_functional = gdt.make_l2_volume_vector_functional(f, rhs_vector, space)

# there are two equivalent ways to restrict the integration domain of the face functional:
# * provide an apply_on_... tag on construction (as done here)
# * provide an apply_on_... tag when appending the functional to the system assembler (bindings not yet present)
l2_neumann_functional = gdt.make_l2_face_vector_functional(g_N, rhs_vector, space, apply_on_neumann_boundary)

# to handle the Dirichlet boundary we require two ingredients
# * a projection of the boundary values onto the space
# * a collection of the degrees of freedom associated with the boundary, to constrain the resulting linera system
g_D_h_vector = la.IstlDenseVectorDouble(space.size(), 0.)
g_D_h = gdt.make_discrete_function(space, g_D_h_vector, 'dirichlet_projection')
dirichlet_projection_operator = gdt.make_localizable_dirichlet_projection_operator(boundary_info, g_D, g_D_h)
dirichlet_constraints = gdt.make_dirichlet_constraints(boundary_info, space.size(), True)

# compute everything in one grid walk
system_assembler = gdt.make_system_assembler(space)
system_assembler.append(elliptic_operator)
system_assembler.append(l2_force_functional)
system_assembler.append(l2_neumann_functional)
system_assembler.append(dirichlet_projection_operator)
system_assembler.append(dirichlet_constraints)
system_assembler.assemble()

# to form the linear system
# * substract the Dirichlet shift
system_matrix = elliptic_operator.matrix()
rhs_vector -= system_matrix*g_D_h_vector
# * apply the Dirichlet constraints
dirichlet_constraints.apply(system_matrix, rhs_vector)

# solve the linear system
u_h = la.IstlDenseVectorDouble(space.size(), 0.0)
solver = la.make_solver(system_matrix)
# there are three ways to solve the linear system, given a solver
# (i) use the black box variant
solver.apply(rhs_vector, u_h)
# (ii) select the type of solver
#print('available linear solvers:')
#for tp in solver.types():
#    print('  {}'.format(tp))
#solver.apply(rhs_vector, u_h, 'superlu')
# (iii) select the type of solver and its options
#print('options for bicgstab.amg.ssor solver:')
#amg_opts = solver.options('bicgstab.amg.ssor')
#for kk, vv in amg_opts.items():
#    print('  {}: {}'.format(kk, vv))
#amg_opts['precision'] = '1e-8'
#solver.apply(rhs_vector, u_h, amg_opts)

# add the Dirichlet shift
u_h += g_D_h_vector

# visualize (this will write cgfem_esv2007_solution.vtu)
gdt.make_discrete_function(space, u_h, 'solution').visualize('../cgfem_esv2007_solution')